In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark2 = SparkSession.builder.appName('ml').getOrCreate()
#Create a Spark Session
SpSession = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("ml") \
    .config("spark.executor.memory", "0.1g") \
    .config("spark.cores.max","2") \
    .config("spark.sql.warehouse.dir", "/home/sushant/Projects/Spark_Project/temp")\
    .getOrCreate()
    
#Get the Spark Context from Spark Session    
SpContext = SpSession.sparkContext

In [2]:

#Load the data file in ALS format (user, item, rating)
ratingsData = SpContext.textFile("UserItemData.txt")
ratingsData.collect()

#Convert the strings into a proper vector
ratingVector=ratingsData.map(lambda l: l.split(','))\
        .map(lambda l:(int(l[0]), int(l[1]), float(l[2])))

#Build a SQL Dataframe
ratingsDf=SpSession.createDataFrame(ratingVector, \
            ["user","item","rating"])

In [3]:

#build the model based on ALS
from pyspark.ml.recommendation import ALS
als = ALS(rank=10, maxIter=5)
model = als.fit(ratingsDf)

model.userFactors.orderBy("id").collect()

#Create a test data set of users and items you want ratings for
testDf = SpSession.createDataFrame(   \
        [(1001, 9003),(1001,9004),(1001,9005)], \
        ["user","item"])

#Predict            
predictions=(model.transform(testDf).collect())
predictions


[Row(user=1001, item=9004, prediction=-0.6144895553588867),
 Row(user=1001, item=9005, prediction=-3.166208505630493),
 Row(user=1001, item=9003, prediction=8.998315811157227)]